In [1]:
import pickle
import os
import copy
import re
import time
import importlib

import numpy as np

In [2]:
import random

In [3]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)

In [4]:
structure_DB

,init_structure,size
0,"(Atom('Al', [0.0, 0.0, 0.0], index=0))",1
1,"(Atom('Co', [0.0, 0.0, 0.0], index=0))",1
2,"(Atom('Cr', [0.0, 0.0, 0.0], index=0))",1
3,"(Atom('Cu', [0.0, 0.0, 0.0], index=0))",1
4,"(Atom('Fe', [0.0, 0.0, 0.0], index=0))",1
...,...,...
509933,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",10
509934,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",10
509935,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",10
509936,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('C...",10


In [5]:
def delete_files(file_list):
    """Delete multiple files."""
    for file_path in file_list:
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except PermissionError:
            print(f"Permission denied: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

In [6]:
import disall

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
importlib.reload(disall)

<module 'disall' (<_frozen_importlib_external._NamespaceLoader object at 0x14a4c103d570>)>

In [9]:
from disall.grid_run import run_packet
from disall.pyiron_calculator import VASP_pyiron_calculator, read_unfinished_job, _zip_job_dir

2025-01-31 22:06:43,948 - pyiron_log - WARNING - pyiron found a 'templates' folder in the /scratch/group/arroyave_lab/guillermo.vazquez/pyiron_dirs/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().


## how to calculate vasp relaxer
it needs to be an starter or checker.
if not started then it starts it, if running, checks. Maybe different functions

In [10]:
# df = structure_DB.copy(deep = True)

In [11]:
with open('first_reread.pkl', 'rb') as f:
    df = pickle.load(f)

In [12]:
# df [df['first_00'].isnull()]

In [13]:
# delete_files(['first_00.log','first_00.pkl'])
# !rm -r first_00

In [14]:
run_packet?

Signature:
run_packet(
    df_name,
    df_global,
    relaxer_dict,
    input_data=None,
    num_cores=10,
    max_tries=5,
    **kwargs,
)
Docstring:
why we shouldn't run two of these at the same time?
if the numbers are repeated they would be looping the same numbers flagging a job node as occupied when 
two run_packet shouldn't never be run at the same time so the database doesn't handle 
two different loops saving and loading! (only to o when a changed occured?)
File:      /scratch/user/guillermo.vazquez/SAVE/octonary/disall/disall/grid_run.py
Type:      function

In [15]:
input_data = {'-INCAR-NCORE': 16,
    '-INCAR-LORBIT': 1,
    '-INCAR-NSW': 200,
    '-INCAR-PREC': 'Accurate',
    '-INCAR-IBRION': 2,
    '-INCAR-ISMEAR': 0,
    '-INCAR-SIGMA': 0.02,
    '-INCAR-ENCUT': 400,
    '-INCAR-EDIFF': 1E-5,
    '-INCAR-ISIF': 3,
    '-INCAR-ISYM': 0,
    'KPPA': 4000,
    '-INCAR-ISPIN': 2,
    }

In [16]:
relaxer_dic_timeout = {'calculator_label': 'first_00',
                            'relaxer': VASP_pyiron_calculator}

In [17]:
ids = np.arange(980).tolist()

In [18]:
df.loc[ids]

,init_structure,size,first_00
0,"(Atom('Al', [0.0, 0.0, 0.0], index=0))",1,"{'master_status': 'FINISHED', 'error': '', 'st..."
1,"(Atom('Co', [0.0, 0.0, 0.0], index=0))",1,"{'master_status': 'FINISHED', 'error': '', 'st..."
2,"(Atom('Cr', [0.0, 0.0, 0.0], index=0))",1,"{'master_status': 'FINISHED', 'error': '', 'st..."
3,"(Atom('Cu', [0.0, 0.0, 0.0], index=0))",1,"{'master_status': 'FINISHED', 'error': '', 'st..."
4,"(Atom('Fe', [0.0, 0.0, 0.0], index=0))",1,"{'master_status': 'FINISHED', 'error': '', 'st..."
...,...,...,...
975,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('N...",4,"{'master_status': 'FINISHED', 'job_name': 'vas..."
976,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('N...",4,"{'master_status': 'FINISHED', 'job_name': 'vas..."
977,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('N...",4,"{'master_status': 'FINISHED', 'job_name': 'vas..."
978,"(Atom('Cu', [0.0, 0.0, 0.0], index=0), Atom('N...",4,"{'master_status': 'FINISHED', 'job_name': 'vas..."


In [19]:
# df = run_packet('first_reread',
#         df,
#         relaxer_dic_timeout,
#         input_data = copy.deepcopy(input_data),
#         id_list = ids,
#         num_cores = 980,
#         max_tries = 5)

In [20]:
# df = run_packet('first_reread',
#         df,
#         relaxer_dic_timeout,
#         input_data = copy.deepcopy(input_data),
#         id_list = ids,
#         num_cores = 20,
#         max_tries = 5)

In [74]:
ids = np.arange(1000,1100).tolist()
for i in range(20):
    try:
        df = run_packet('first_reread',
                df,
                relaxer_dic_timeout,
                input_data = copy.deepcopy(input_data),
                id_list = ids,
                num_cores = 20,
                max_tries = 5)
        time.sleep(30)
    except Exception as e:
        print('EXCEPTION : ', e)
        time.sleep(30)

in 20 number of cores
##########< i: 0, to_calculate[i]: 1000  >##########
this calculation has finished succesfully, already parsed
##########< i: 1, to_calculate[i]: 1001  >##########
this calculation has finished succesfully, already parsed
##########< i: 2, to_calculate[i]: 1002  >##########
this calculation has finished succesfully, already parsed
##########< i: 3, to_calculate[i]: 1003  >##########
this calculation has finished succesfully, already parsed
##########< i: 4, to_calculate[i]: 1004  >##########


2025-01-30 09:36:45,002 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1976301
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 5, to_calculate[i]: 1005  >##########
this calculation has finished succesfully, already parsed
##########< i: 6, to_calculate[i]: 1006  >##########
this calculation has finished succesfully, already parsed
##########< i: 7, to_calculate[i]: 1007  >##########
this calculation has finished succesfully, already parsed
##########< i: 8, to_calculate[i]: 1008  >##########
this calculation has finished succesfully, already parsed
##########< i: 9, to_calculate[i]: 1009  >##########
this calculation has finished succesfully, already parsed
##########< i: 10, to_calculate[i]: 1010  >##########
this calculation has finished succesfully, already parsed
##########< i: 11, to_calculate[i]: 1011  >##########
this calculation has finished succesfully, already parse

2025-01-30 09:36:46,053 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1976509
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 45, to_calculate[i]: 1045  >##########
pyiron_status start of loop: running
pyiron QID 1976639
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 46, to_calculate[i]: 1046  >##########
pyiron_status start of loop: running
pyiron QID 1976640
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 47, to_calculate[i]: 1047  >##########
pyiron_status start of loop: running
pyiron QID 1976641
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 48, to_calculate[i]: 1048  >##########
pyiron_status start of loop: running
pyiron QID 1976642
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 49, to_calculate[i]: 1049  >##########
pyiron_stat

...

pyiron_status start of loop: finished
pyiron QID 1976665
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 70, to_calculate[i]: 1070  >##########
pyiron_status start of loop: running
pyiron QID 1976666
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 71, to_calculate[i]: 1071  >##########
pyiron_status start of loop: running
pyiron QID 1976667
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 72, to_calculate[i]: 1072  >##########
this calculation has finished succesfully, already parsed
##########< i: 73, to_calculate[i]: 1073  >##########
pyiron_status start of loop: running
pyiron QID 1976670
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 74, to_calculate[i]: 1074  >##########


2025-01-30 09:54:27,342 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1976672
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 75, to_calculate[i]: 1075  >##########
pyiron_status start of loop: running
pyiron QID 1976673
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 76, to_calculate[i]: 1076  >##########
pyiron_status start of loop: running
pyiron QID 1976674
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 77, to_calculate[i]: 1077  >##########
pyiron_status start of loop: running
pyiron QID 1976675
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 78, to_calculate[i]: 1078  >##########
pyiron_status start of loop: running
pyiron QID 1976676
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 79, to_calculate[i]: 1079  >##########
pyiron_stat

2025-01-30 09:54:35,451 - pyiron_log - WARNING - Job aborted


EXCEPTION :  run_queue.sh crashed


In [25]:
ids = np.arange(1000,1100).tolist()
for i in range(1):
    try:
        df = run_packet('first_reread',
                df,
                relaxer_dic_timeout,
                input_data = copy.deepcopy(input_data),
                id_list = ids,
                num_cores = 20,
                max_tries = 5)
        time.sleep(30)
    except Exception as e:
        print('EXCEPTION : ', e)
        time.sleep(30)

in 20 number of cores
##########< i: 0, to_calculate[i]: 1000  >##########
this calculation has finished succesfully, already parsed
##########< i: 1, to_calculate[i]: 1001  >##########
this calculation has finished succesfully, already parsed
##########< i: 2, to_calculate[i]: 1002  >##########
this calculation has finished succesfully, already parsed
##########< i: 3, to_calculate[i]: 1003  >##########
this calculation has finished succesfully, already parsed
##########< i: 4, to_calculate[i]: 1004  >##########
this calculation has finished succesfully, already parsed
##########< i: 5, to_calculate[i]: 1005  >##########
this calculation has finished succesfully, already parsed
##########< i: 6, to_calculate[i]: 1006  >##########
this calculation has finished succesfully, already parsed
##########< i: 7, to_calculate[i]: 1007  >##########
this calculation has finished succesfully, already parsed
##########< i: 8, to_calculate[i]: 1008  >##########
this calculation has finished succesf

2025-01-30 10:36:20,475 - pyiron_log - WARNING - Could not access indices, returning None!


status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 85, to_calculate[i]: 1085  >##########
this calculation has finished succesfully, already parsed
##########< i: 86, to_calculate[i]: 1086  >##########
this calculation has finished succesfully, already parsed
##########< i: 87, to_calculate[i]: 1087  >##########
this calculation has finished succesfully, already parsed
##########< i: 88, to_calculate[i]: 1088  >##########
this calculation has finished succesfully, already parsed
##########< i: 89, to_calculate[i]: 1089  >##########
this calculation has finished succesfully, already parsed
##########< i: 90, to_calculate[i]: 1090  >##########
this calculation has finished succesfully, already parsed
##########< i: 91, to_calculate[i]: 1091  >##########
this calculation has finished succesfully, already parsed
##########< i: 92, to_calculate[i]: 1092  >#######

In [22]:
# unmarked_at_4 = np.arange(df.attrs['first_00']['marked'][-1] + 1, df[df['size'] == 4].index.tolist()[-1] + 1).tolist()

In [23]:
# np.arange(1100, df[df['size'] == 4].index.tolist()[-1] + 1)

In [29]:
# df.to_pickle('df_1100'+'.pkl')

In [18]:
unmarked_at_4 = np.arange(1100, df[df['size'] == 4].index.tolist()[-1] + 1).tolist()

In [19]:
random.seed(42)
test_ids = random.sample(unmarked_at_4, 500)

In [20]:
test_ids.sort()

In [21]:

for i in range(1):
    try:
        df = run_packet('first_reread',
                df,
                relaxer_dic_timeout,
                input_data = copy.deepcopy(input_data),
                id_list = test_ids,
                num_cores = 20,
                max_tries = 5)
        time.sleep(30)
    except Exception as e:
        print('EXCEPTION : ', e)
        time.sleep(30)

in 20 number of cores
##########< i: 0, to_calculate[i]: 1102  >##########
this calculation has finished succesfully, already parsed
##########< i: 1, to_calculate[i]: 1108  >##########
this calculation has finished succesfully, already parsed
##########< i: 2, to_calculate[i]: 1113  >##########
this calculation has finished succesfully, already parsed
##########< i: 3, to_calculate[i]: 1114  >##########
this calculation has finished succesfully, already parsed
##########< i: 4, to_calculate[i]: 1126  >##########
this calculation has finished succesfully, already parsed
##########< i: 5, to_calculate[i]: 1129  >##########
this calculation has finished succesfully, already parsed
##########< i: 6, to_calculate[i]: 1132  >##########
this calculation has finished succesfully, already parsed
##########< i: 7, to_calculate[i]: 1138  >##########
this calculation has finished succesfully, already parsed
##########< i: 8, to_calculate[i]: 1139  >##########
this calculation has finished succesf

In [17]:
unmarked_at_5 = df[df['size'] == 5].index.tolist()
random.seed(42)
ids_5 = random.sample(unmarked_at_5, 500)
ids_5.sort()

In [20]:
for i in range(1):
    try:
        df = run_packet('first_reread',
                df,
                relaxer_dic_timeout,
                input_data = copy.deepcopy(input_data),
                id_list = ids_5,
                num_cores = 20,
                max_tries = 5)
        time.sleep(30)
    except Exception as e:
        print('EXCEPTION : ', e)
        time.sleep(30)

in 20 number of cores
##########< i: 0, to_calculate[i]: 4455  >##########
this calculation has finished succesfully, already parsed
##########< i: 1, to_calculate[i]: 4481  >##########
this calculation has finished succesfully, already parsed
##########< i: 2, to_calculate[i]: 4499  >##########
this calculation has finished succesfully, already parsed
##########< i: 3, to_calculate[i]: 4552  >##########
this calculation has finished succesfully, already parsed
##########< i: 4, to_calculate[i]: 4563  >##########
this calculation has finished succesfully, already parsed
##########< i: 5, to_calculate[i]: 4598  >##########
this calculation has finished succesfully, already parsed
##########< i: 6, to_calculate[i]: 4634  >##########
this calculation has finished succesfully, already parsed
##########< i: 7, to_calculate[i]: 4687  >##########
this calculation has finished succesfully, already parsed
##########< i: 8, to_calculate[i]: 4765  >##########
this calculation has finished succesf

2025-02-01 00:49:40,116 - pyiron_log - WARNING - Could not access indices, returning None!


master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 17, to_calculate[i]: 5157  >##########
this calculation has finished succesfully, already parsed
##########< i: 18, to_calculate[i]: 5196  >##########
this calculation has finished succesfully, already parsed
##########< i: 19, to_calculate[i]: 5217  >##########
this calculation has finished succesfully, already parsed
##########< i: 20, to_calculate[i]: 5267  >##########
this calculation has finished succesfully, already parsed
##########< i: 21, to_calculate[i]: 5274  >##########
this calculation has finished succesfully, already parsed
##########< i: 22, to_calculate[i]: 5333  >##########
this calculation has finished succesfully, already parsed
##########< i: 23, to_calculate[i]: 5362  >##########
this calculation has finished succesfully, already parsed
##########< i: 24, to_calculate[i]: 5377  >##########
this calculation has finished succesfully, already parsed
#########

2025-02-01 00:49:40,872 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1985859
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 43, to_calculate[i]: 5615  >##########
this calculation has finished succesfully, already parsed
##########< i: 44, to_calculate[i]: 5634  >##########


...

pyiron_status start of loop: finished
pyiron QID 1985881
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 65, to_calculate[i]: 6120  >##########


2025-02-01 00:49:49,862 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1985882
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 66, to_calculate[i]: 6125  >##########


2025-02-01 00:49:50,384 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1985883
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 67, to_calculate[i]: 6189  >##########
pyiron_status start of loop: initialized
PASS to RUN --> id000000000006189
SENDING...
The job vasp_000000000006189 was saved and received the ID: 30009
Queue system id:  1986046
##########< i: 68, to_calculate[i]: 6236  >##########
pyiron_status start of loop: initialized
PASS to RUN --> id000000000006236
SENDING...
The job vasp_000000000006236 was saved and received the ID: 30010
Queue system id:  1986047
##########< i: 69, to_calculate[i]: 6242  >##########
pyiron_status start of loop: initialized
PASS to RUN --> id000000000006242
SENDING...
The job vasp_000000000006242 was saved and received the ID: 30011
Queue system id:  1986048
##########< i: 70, to_calculate[i]: 6270  >##########
pyiron_status start of loop

In [ ]:
for i in range(400):
    try:
        df = run_packet('first_reread',
                df,
                relaxer_dic_timeout,
                input_data = copy.deepcopy(input_data),
                id_list = ids_5,
                num_cores = 25,
                max_tries = 5)
        time.sleep(30)
    except Exception as e:
        print('EXCEPTION : ', e)
        time.sleep(30)

in 25 number of cores
##########< i: 0, to_calculate[i]: 4455  >##########
this calculation has finished succesfully, already parsed
##########< i: 1, to_calculate[i]: 4481  >##########
this calculation has finished succesfully, already parsed
##########< i: 2, to_calculate[i]: 4499  >##########
this calculation has finished succesfully, already parsed
##########< i: 3, to_calculate[i]: 4552  >##########
this calculation has finished succesfully, already parsed
##########< i: 4, to_calculate[i]: 4563  >##########
this calculation has finished succesfully, already parsed
##########< i: 5, to_calculate[i]: 4598  >##########
this calculation has finished succesfully, already parsed
##########< i: 6, to_calculate[i]: 4634  >##########
this calculation has finished succesfully, already parsed
##########< i: 7, to_calculate[i]: 4687  >##########
this calculation has finished succesfully, already parsed
##########< i: 8, to_calculate[i]: 4765  >##########
this calculation has finished succesf

2025-02-01 02:00:05,611 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1986558
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 132, to_calculate[i]: 8216  >##########
pyiron_status start of loop: finished
pyiron QID None
SQUEUE ERROR 0
list index out of range
SQUEUE ERROR 1
list index out of range
SQUEUE ERROR 2
list index out of range
finished/aborted alternate squeue check, status: finished


2025-02-01 02:00:06,781 - pyiron_log - WARNING - Could not access indices, returning None!


alternate result, QID: 1986559, status_slurm: COMPLETED, run_time: 00:13:18
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 133, to_calculate[i]: 8257  >##########
pyiron_status start of loop: running
pyiron QID 1986560
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 134, to_calculate[i]: 8260  >##########
pyiron_status start of loop: running
pyiron QID 1986561
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 135, to_calculate[i]: 8299  >##########
pyiron_status start of loop: running
pyiron QID 1986562
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 136, to_calculate[i]: 8318  >##########
pyiron_status start of loop: running
pyiron QID 1986563
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 137, to_calculate[i]: 8345 

...

pyiron_status start of loop: finished
pyiron QID 1986797
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 261, to_calculate[i]: 12647  >##########
pyiron_status start of loop: running
pyiron QID 1986798
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 262, to_calculate[i]: 12684  >##########
pyiron_status start of loop: running
pyiron QID 1986799
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 263, to_calculate[i]: 12725  >##########
pyiron_status start of loop: running
pyiron QID 1986800
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 264, to_calculate[i]: 12734  >##########
pyiron_status start of loop: running
pyiron QID 1986801
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 265, to_calculate[i]: 12763  >##########
p

2025-02-01 03:40:37,594 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1986790
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 255, to_calculate[i]: 12402  >##########


2025-02-01 03:40:38,067 - pyiron_log - WARNING - Could not access indices, returning None!


pyiron_status start of loop: finished
pyiron QID 1986791
status_pyiron: finished, status_slurm: COMPLETED
parsing pyiron job...
master_status FINISHED
this calculation has finished succesfully, parsing master dictionary...
##########< i: 256, to_calculate[i]: 12408  >##########
pyiron_status start of loop: running
pyiron QID 1986792
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 257, to_calculate[i]: 12419  >##########
pyiron_status start of loop: running
pyiron QID 1986793
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 258, to_calculate[i]: 12451  >##########
pyiron_status start of loop: running
pyiron QID 1986795
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 259, to_calculate[i]: 12531  >##########
pyiron_status start of loop: running
pyiron QID 1986796
status_pyiron: running, status_slurm: RUNNING
master_status RUNNING
##########< i: 260, to_calculate[i]: 12625  >##########
t